In [39]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
options = ['hit', 'stand']
def random_policy():
    return random.choice(options)

# cards
suits = ["Spades", "Hearts", "Clubs", "Diamonds"]
cards = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
cards_values = {"A": 11, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "10":10, "J":10, "Q":10, "K":10}

# suit value 
suits_values = {"Spades":"\u2664", "Hearts":"\u2661", "Clubs": "\u2667", "Diamonds": "\u2662"}

class Card: 
    def __init__(self, suit, value, card_value):
        self.suit = suit
        self.value = value
        self.card_value = card_value

deck = []
def shuffle_deck():
    # The deck of cards
    global deck
    deck = []

    for suit in suits: # Loop for every type of suit
        for card in cards: # Loop for every type of card in a suit
            # Adding card to the deck
            deck.append(Card(suits_values[suit], card, cards_values[card]))

In [3]:
# Creating Q-Table
# 23 rows: 1 for each state, 3 columns for state, action, q-value
def init_q_table():
    global q_table 
    q_table = {'win':{'stand':1000},
               'loss':{'stand':-1000},
               '1':{'hit':0,'stand':0},
               '2':{'hit':0,'stand':0},
               '3':{'hit':0,'stand':0},
               '4':{'hit':0,'stand':0},
               '5':{'hit':0,'stand':0},
               '6':{'hit':0,'stand':0},
               '7':{'hit':0,'stand':0},
               '8':{'hit':0,'stand':0},
               '9':{'hit':0,'stand':0},
               '10':{'hit':0,'stand':0},
               '11':{'hit':0,'stand':0},
               '12':{'hit':0,'stand':0},
               '13':{'hit':0,'stand':0},
               '14':{'hit':0,'stand':0},
               '15':{'hit':0,'stand':0},
               '16':{'hit':0,'stand':0},
               '17':{'hit':0,'stand':0},
               '18':{'hit':0,'stand':0},
               '19':{'hit':0,'stand':0},
               '20':{'hit':0,'stand':0},
               '21':{'hit':0,'stand':0},
              }
# Policy Table
policy_table = {'win':'stand',
                'loss':'stand',
                '1':'hit',
                '2':{'hit':0,'stand':0},
                '3':{'hit':0,'stand':0},
                '4':{'hit':0,'stand':0},
                '5':{'hit':0,'stand':0},
                '6':{'hit':0,'stand':0},
                '7':{'hit':0,'stand':0},
                '8':{'hit':0,'stand':0},
                '9':{'hit':0,'stand':0},
                '10':{'hit':0,'stand':0},
                '11':{'hit':0,'stand':0},
                '12':{'hit':0,'stand':0},
                '13':{'hit':0,'stand':0},
                '14':{'hit':0,'stand':0},
                '15':{'hit':0,'stand':0},
                '16':{'hit':0,'stand':0},
                '17':{'hit':0,'stand':0},
                '18':{'hit':0,'stand':0},
                '19':{'hit':0,'stand':0},
                '20':{'hit':0,'stand':0},
                '21':{'hit':0,'stand':0},
              }

In [11]:
def rewards(current_state):
    if current_state == 'win':
        return 1000
    elif current_state == 'loss':
        return -1000
    return int(current_state)

def choose_action(current_state, epi):
    t = np.random.uniform()
    
    if t < epi:
        #randomly choose from all possible actions
        action = random_policy()  
    if t >= epi:
        #action with highest q value for current state
        hit = q_table[str(current_state)]['hit']
        stand = q_table[str(current_state)]['stand']
        action = 'hit' if hit >= stand else 'stand'
        
    return action

def update_q_table(current_state, action, next_state):
    global q_table
    #= (1-.1)  q[s,a] + .1(r(s) + .1q[s',a']) 
   
    if (current_state != 'win' and current_state != 'loss'):
        max_q_next = max(q_table[next_state].values())  #find the max q value for next state
        qval_current = (1-.1) * q_table[current_state][action] # q_value for current state
        qval_next = .1 * (rewards(current_state) + .1 * (max_q_next)) # rewards + max q_value for next state
        q_table[current_state][action] += qval_current + qval_next

In [19]:
# Function for a single game of blackjack
def blackjack_game():
    shuffle_deck()
    
    player_cards = []
    dealer_cards = []
    
    player_score = 0
    dealer_score = 0
    
    # ======= DEALER'S HAND FIRST ======= #
   
    # DEALING DEALER'S HAND
    while len (dealer_cards) < 2:          
        dealer_card = random.choice(deck)
        dealer_cards.append(dealer_card)
        deck.remove(dealer_card)
        dealer_score += dealer_card.card_value
        
    # DEALER: if both cards are aces, make an ace == 1 
    if len(dealer_cards) == 2:
        if dealer_cards[0].card_value == 11 and dealer_cards[1].card_value == 11:
            dealer_cards[1].card_value = 1
            dealer_score -= 10
        
    # LOOP FOR DEALER TO GET MORE CARDS
    while dealer_score < 21:
        dealer_card = random.choice(deck)
        predicted_value = dealer_card.card_value + dealer_score

        if predicted_value > 21 and dealer_card.card_value == 11:
            predicted_value -=10
        
        if predicted_value > 21:
            break
        else:
            dealer_score = predicted_value  
            
    # ======= DEALER'S HAND END ======= #   
        
    # ======= PLAYER'S HAND SECOND ======= #
    
    # DEALING PLAYER'S HAND
    while len (player_cards) < 2:    
        player_card = random.choice(deck)
        player_cards.append(player_card)
        deck.remove(player_card)
        player_score += player_card.card_value      
    
    # PLAYER: if both cards are aces, make an ace == 1 
    if len(player_cards) == 2:
        if player_cards[0].card_value == 11 and player_cards[1].card_value == 11:
            player_cards[0].card_value = 1
            player_score -= 10
    
    choice = 'hit'
    prev_score = player_score
    # LOOP FOR PLAYER TO GET MORE CARDS
    while player_score < 21 and choice != 'stand':
        choice = choose_action(player_score, .1)
        
        if choice == 'hit':
            # deal another new card
            player_card = random.choice(deck)
            player_cards.append(player_card)
            deck.remove(player_card)
 
            # Updating player score   
            player_score += player_card.card_value
            
            # if the player "busts" but has an ace, convert
            c = 0
            while player_score > 21 and c < len(player_cards):
                if player_cards[c].card_value == 11:
                    player_cards[c].card_value = 1
                    player_score -= 10
                    c += 1
                else:
                    c += 1 
            
        # updating q_table 
        if player_score <= 21:    
            update_q_table(str(prev_score), choice, str(player_score))
            prev_score = player_score
            
    choice = 'stand'
    # ======= PLAYER'S HAND END ======= #
    
    # ======= CHECK WINNING SCORE ======= #
    
    
    if player_score == 21 or dealer_score > 21: # check player blackjack, dealer bust shouldn't happen but just in case
        result = 'win'
    elif player_score >= dealer_score and player_score < 21:  # player_score > 21: check player bust
        result = 'win'
    else:
        result = 'loss'
    
    # updating q_table 
    update_q_table(str(prev_score), choice, result)
    
    #print("dealer score: ", dealer_score)
    #print("prev score: ", prev_score, "choice: ", choice, "player score: ", player_score)
    #print("result", result)
    
    return 

In [30]:
def optimize_policy():
    # init q-table with q-values of 0
    init_q_table()
    for i in range(0, 2000):
        blackjack_game()
        
def construct_policy(q_table):
    #for each state s:
        #find action a with the largest q-value
        #add s, a to the policy table
    global policy_table 
    policy_table = { 'win' : 'stand', 'loss' : 'stand'}
    
    for state in q_table:
        if state != 'win' and state != 'loss':
            if q_table[state]['hit'] >= q_table[state]['stand']:
                policy_table[state] = 'hit'
            else:
                policy_table[state] = 'stand'

In [68]:
df = pd.DataFrame()
for i in range (0, 20):
    optimize_policy()
    construct_policy(q_table)
    df = pd.concat([df, pd.DataFrame(policy_table.values())], ignore_index=True, axis = 1)

In [69]:
df

0      1      2      3      4      5      6      7      8      9   \
0   stand  stand  stand  stand  stand  stand  stand  stand  stand  stand   
1   stand  stand  stand  stand  stand  stand  stand  stand  stand  stand   
2     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
3     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
4     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
5     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
6     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
7     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
8     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
9     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
10    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
11    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
12    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
13    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit   
14    hit    hit    hit    hit  stand    hit    hit    hit    hit    hit   
15    hit  stand    hit    hit    hit    hit    hit  stand    hit    hit   
16    hit    hit  stand    hit  stand    hit    hit    hit    hit    hit   
17  stand  stand    hit    hit  stand  stand    hit    hit    hit    hit   
18    hit  stand  stand  stand  stand  stand  stand  stand  stand    hit   
19  stand  stand  stand    hit  stand  stand  stand    hit  stand    hit   
20  stand  stand  stand  stand  stand  stand    hit  stand  stand    hit   
21  stand  stand  stand  stand  stand  stand  stand  stand  stand  stand   
22  stand  stand  stand  stand  stand  stand  stand  stand  stand  stand   

       10     11     12     13     14     15     16     17     18     19  
0   stand  stand  stand  stand  stand  stand  stand  stand  stand  stand  
1   stand  stand  stand  stand  stand  stand  stand  stand  stand  stand  
2     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
3     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
4     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
5     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
6     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
7     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
8     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
9     hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
10    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
11    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
12    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
13    hit    hit    hit    hit    hit    hit    hit    hit    hit    hit  
14    hit    hit  stand    hit    hit    hit    hit    hit    hit    hit  
15    hit  stand    hit    hit    hit    hit    hit    hit    hit    hit  
16    hit    hit  stand    hit  stand    hit  stand    hit    hit    hit  
17    hit    hit    hit    hit  stand    hit    hit    hit  stand    hit  
18  stand  stand    hit  stand  stand    hit    hit  stand  stand    hit  
19  stand    hit    hit  stand  stand  stand    hit  stand  stand  stand  
20  stand  stand    hit  stand  stand  stand    hit    hit    hit  stand  
21  stand  stand  stand  stand  stand  stand    hit  stand  stand  stand  
22  stand  stand  stand  stand  stand  stand  stand  stand  stand  stand